In [1]:
import pandas as pd
import csv
import pandas.io.gbq as bq

In [8]:
# first find the geartypes

q = '''SELECT
  b.mmsi mmsi,
  b.clav_TUVI clav_TUVI,
  a.rfmo_id as rfmo_id
FROM ( 
  SELECT
    *
  FROM
    [world-fishing-827:Registry_matching_sources.CLAV_12_14_2015]
  WHERE
    rfmo_name ="IATTC") a
INNER JOIN (
  SELECT
    clav_TUVI,
    mmsi
  FROM
    [world-fishing-827:CLAV_match_results.v7_results]) b
ON
  a.clav_TUVI = b.clav_TUVI
  group by mmsi, clav_TUVI, rfmo_id
'''

ffa =pd.io.gbq.read_gbq(q, project_id='world-fishing-827', verbose = False )

In [9]:
ffa = ffa.set_index('mmsi')
ffa.head()

,callsign,tonnage,length,ffa_ENGINE_POWER,ffa_ENGINE_UNIT
mmsi,,,,,
416,BH3233,496.00,52.65,1100,HP
512000,ZMAE,1355.00,61.00,3070,HP
4162316,BJ5024,95.66,23.90,940,hp
8615784,3FXQ4,2856.00,91.02,3660,HP
22412400,EBTL,7207.00,132.90,5850,KW


In [21]:
# 1 ps = 0.735499 kw 
# 1 hp = 0.7457 kw

def calc_kw(a,b):
    try:
        c = 1
        if b.upper() == "HP":
            c = .75
        if b.upper() == "PS":
            c = .74
        return float(a)*c
    except:
        return None


ffa['power'] = ffa.apply(lambda row: calc_kw(row['ffa_ENGINE_POWER'], row['ffa_ENGINE_UNIT']), axis=1)

In [22]:
ffa.head()

,callsign,tonnage,length,ffa_ENGINE_POWER,ffa_ENGINE_UNIT,power
mmsi,,,,,,
416,BH3233,496.00,52.65,1100,HP,825.0
512000,ZMAE,1355.00,61.00,3070,HP,2302.5
4162316,BJ5024,95.66,23.90,940,hp,705.0
8615784,3FXQ4,2856.00,91.02,3660,HP,2745.0
22412400,EBTL,7207.00,132.90,5850,KW,5850.0


In [19]:
ffa[['callsign','tonnage','length','power']].to_csv('input_lists/ffa.csv')

In [23]:
ffa.to_csv('input_lists/ffa.csv')